# Training and Evaluating on the Training set

The goal of this notebook is to see how well models perform the classification just through the training dataset

In [3]:
# Data wrangling libraries
import numpy as np
import pandas as pd
import os

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [4]:
# modelling
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, GridSearchCV, learning_curve
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline

In [5]:
# evaluation metrics libraries
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import (precision_score, recall_score, f1_score,accuracy_score)

In [6]:
# Construct the directory containing the CSV files
current_dir = os.getcwd()  # Get the current working directory
data_dir = os.path.join(current_dir, '..', 'data') 

# List of file names to read
file_names = ['qap_global_df.csv']

# Initialize a dictionary to store DataFrames
dataframes = {}

# Loop through the file names, read each file, and store in the dictionary
for file_name in file_names:
    file_path = os.path.join(data_dir, file_name)  # Construct the full file path
    dataframes[file_name] = pd.read_csv(file_path, index_col=0)  # Read the CSV file and store in the dictionary

# Access the DataFrames using their file names
qap_global_df = dataframes['qap_global_df.csv']

In [7]:
qap_global_df.columns

Index(['sample_id', 'rock_name', 'qap_name', 'major_id', 'method_id', 'sio2',
       'tio2', 'al2o3', 'fe2o3', 'mgo', 'cao', 'mno', 'k2o', 'na2o', 'p2o5'],
      dtype='object')

In [8]:
qap_global_df

,sample_id,rock_name,qap_name,major_id,method_id,sio2,tio2,al2o3,fe2o3,mgo,cao,mno,k2o,na2o,p2o5
10,24,granitoid,quartz diorite,289207,4989,52.030,0.97,18.300,4.260,3.800,8.290,0.13,0.930,3.010,0.200
16,30,kozakura felsic rock,granodiorite,458254,5238,63.640,0.38,15.430,2.160,2.100,4.370,0.14,3.870,3.370,0.250
19,41,trachyte,monzonite,448886,5382,62.860,0.43,18.250,4.250,1.080,0.640,0.08,6.350,5.190,0.070
21,47,hb gabbro,quartz diorite,294667,5254,52.262,0.71,18.418,9.056,8.568,9.193,0.17,0.557,1.354,0.079
22,50,fine px-hbl gabbronorite,quartz monzodiorite,359363,5380,55.930,1.07,15.960,1.950,4.730,7.540,0.19,2.010,1.900,0.200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145535,1022088,charnockite,quartz monzonite,481939,1,65.770,0.55,16.050,1.650,0.270,2.950,0.08,5.630,3.300,0.150
145536,1022089,charnockite,quartz monzonite,443128,1,62.390,0.78,16.130,2.790,0.460,3.810,0.11,5.120,3.310,0.230
145537,1022090,charnockite,monzogranite,422311,1,60.710,1.09,13.890,4.290,0.410,4.400,0.20,4.370,2.830,0.300
145538,1022091,charnockite,quartz monzonite,459517,1,63.750,0.69,16.620,2.090,0.280,3.510,0.07,5.570,3.240,0.200


In [9]:
# load the data
X = qap_global_df[['sio2', 'tio2', 'al2o3', 'fe2o3', 'mgo', 'cao', 'mno', 'k2o', 'na2o', 'p2o5']] # na2o is still included
y = qap_global_df['qap_name']

In [10]:
# split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=13, stratify=y)

In [11]:
# Initialize StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=13)

In [21]:
# initialize the random forest classifier
rf = RandomForestClassifier(random_state=13)

In [34]:
# Set numpy print options to avoid line wrapping
np.set_printoptions(threshold=np.inf, linewidth=np.inf)  # Show entire confusion matrix in one line

# Cross-validation loop
fold = 1
for train_index, val_index in skf.split(X_train, y_train):
    X_train_cv, X_val_cv = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_cv, y_val_cv = y_train.iloc[train_index], y_train.iloc[val_index]
    
    # Train the classifier
    rf.fit(X_train_cv, y_train_cv)
    
    # Make predictions
    y_val_pred = rf.predict(X_val_cv)
    
    # Print classification report for this fold
    print(f"\nFold {fold}")
    print(classification_report(y_val_cv, y_val_pred, zero_division=0))
    
    # Compute confusion matrix
    cm = confusion_matrix(y_val_cv, y_val_pred)
    
    # Print confusion matrix
    print("Confusion Matrix:")
    print(cm)
    
    fold += 1


Fold 1
                                precision    recall  f1-score   support

       alkali feldspar granite       0.97      0.78      0.86        40
alkali feldspar quartz syenite       0.80      0.95      0.87        39
       alkali feldspar syenite       0.00      0.00      0.00         8
                       diorite       0.85      0.78      0.81       414
                        gabbro       0.98      0.99      0.99       790
                  granodiorite       0.93      0.93      0.93      2350
                  monzodiorite       0.81      0.58      0.67       245
                  monzogranite       0.97      0.97      0.97      2923
                     monzonite       0.85      0.61      0.71        87
                quartz diorite       0.85      0.90      0.88       790
           quartz monzodiorite       0.83      0.90      0.86      1267
              quartz monzonite       0.80      0.80      0.80       293
                quartz syenite       0.59      0.50    

In [44]:
# !pip install pyrolite

In [50]:
import pyrolite.comp

In [64]:
# Add a small constant to avoid log(0)
epsilon = 1e-6  # Small constant to avoid log(0)
data_adjusted = X + epsilon

In [68]:
if (X <= 0).any().any():
    raise ValueError("Data contains negative values or zeros, which are invalid for log transformation.")

ValueError: Data contains negative values or zeros, which are invalid for log transformation.

In [76]:
# Check for negative values or zeros
if (X <= 0).any().any():
    print("Data contains negative values or zeros, which are invalid for log transformation.")
    print("Rows and columns with invalid values:")
    print(X[(X <= 0).any(axis=1)])
    
# Add a small constant to avoid log(0)
epsilon = 1  # Small constant to avoid log(0)
data_adjusted = X + epsilon

# Ensure all values are positive after adjustment
if (data_adjusted <= 0).any().any():
    raise ValueError("Data contains non-positive values even after adjustment.")

# Perform centered log-ratio (CLR) transformation
clr_data = clr(data_adjusted)

print(clr_data)

Data contains negative values or zeros, which are invalid for log transformation.
Rows and columns with invalid values:
         sio2  tio2  al2o3  fe2o3   mgo   cao   mno   k2o  na2o  p2o5
328     76.52  0.05  12.79   1.27  0.03  0.66  0.03  4.88  3.54 -0.01
329     76.88  0.03  12.79   1.28  0.03  0.64  0.03  4.11  4.15 -0.01
330     76.74  0.04  13.02   1.07  0.05  0.67  0.03  4.84  3.60 -0.01
331     77.38  0.05  12.67   0.98  0.04  0.61  0.02  4.54  3.76 -0.01
332     76.74  0.03  12.65   0.90  0.04  0.60  0.02  5.07  3.34 -0.01
...       ...   ...    ...    ...   ...   ...   ...   ...   ...   ...
145486  64.09  0.24  16.11   6.02  0.00  0.09  0.13  5.88  7.48  0.03
145503  69.63  0.19  13.98   4.61  0.03  0.43  0.13  4.64  6.36  0.00
145504  71.85  0.21  12.28   4.65  0.00  0.46  0.06  4.46  6.02  0.00
145509  74.69  0.19  11.44   3.57  0.01  0.22  0.09  4.34  5.45  0.00
145515  67.65  0.29  12.40   5.52  0.00  1.44  0.20  4.72  6.73  0.06

[1562 rows x 10 columns]


ValueError: Data contains non-positive values even after adjustment.

In [66]:
lr_df = data_adjusted.pyrocomp.CLR()  # using a centred log-ratio transformation

C:\Users\MANCY036\Anaconda3\lib\site-packages\pyrolite\comp\codata.py:177: RuntimeWarning: invalid value encountered in log
  Y = np.log(X)  # Log operation


In [62]:
lr_df

,CLR(sio2/G),CLR(tio2/G),CLR(al2o3/G),CLR(fe2o3/G),CLR(mgo/G),CLR(cao/G),CLR(mno/G),CLR(k2o/G),CLR(na2o/G),CLR(p2o5/G)
10,3.041091,-0.941189,1.996172,0.538540,0.424272,1.204321,-2.950950,-0.983300,0.191211,-2.520167
16,3.340788,-1.780038,1.923860,-0.042346,-0.070517,0.662309,-2.778567,0.540800,0.402459,-2.198748
19,3.582020,-1.402860,2.345275,0.888029,-0.481929,-1.005177,-3.084618,1.289565,1.087844,-3.218150
21,3.105807,-1.192952,2.062866,1.352965,1.297572,1.367980,-2.622419,-1.435652,-0.547399,-3.388769
22,3.106737,-0.849705,1.852721,-0.249535,0.636561,1.102858,-2.578095,-0.219229,-0.275510,-2.526802
...,...,...,...,...,...,...,...,...,...,...
145535,3.672517,-1.111484,2.262062,-0.012871,-1.822980,0.568159,-3.039375,1.214463,0.680276,-2.410767
145536,3.392813,-0.989053,2.040089,0.285450,-1.517121,0.597037,-2.947867,0.892563,0.456356,-2.210268
145537,3.248971,-0.770960,1.774031,0.599149,-1.748736,0.624467,-2.466576,0.617625,0.183139,-2.061111
145538,3.561109,-0.964923,2.216747,0.143304,-1.866825,0.661756,-3.253120,1.123535,0.581714,-2.203298


In [78]:
X

,sio2,tio2,al2o3,fe2o3,mgo,cao,mno,k2o,na2o,p2o5
10,52.030,0.97,18.300,4.260,3.800,8.290,0.13,0.930,3.010,0.200
16,63.640,0.38,15.430,2.160,2.100,4.370,0.14,3.870,3.370,0.250
19,62.860,0.43,18.250,4.250,1.080,0.640,0.08,6.350,5.190,0.070
21,52.262,0.71,18.418,9.056,8.568,9.193,0.17,0.557,1.354,0.079
22,55.930,1.07,15.960,1.950,4.730,7.540,0.19,2.010,1.900,0.200
...,...,...,...,...,...,...,...,...,...,...
145535,65.770,0.55,16.050,1.650,0.270,2.950,0.08,5.630,3.300,0.150
145536,62.390,0.78,16.130,2.790,0.460,3.810,0.11,5.120,3.310,0.230
145537,60.710,1.09,13.890,4.290,0.410,4.400,0.20,4.370,2.830,0.300
145538,63.750,0.69,16.620,2.090,0.280,3.510,0.07,5.570,3.240,0.200


In [80]:
X.describe()

,sio2,tio2,al2o3,fe2o3,mgo,cao,mno,k2o,na2o,p2o5
count,69353.000000,69353.000000,69353.000000,69353.000000,69353.000000,69353.000000,69353.000000,69353.000000,69353.000000,69353.000000
mean,62.732156,0.835592,14.541221,3.110208,3.078008,4.684103,0.105232,2.947538,3.354250,0.213567
std,10.582696,0.936746,2.365244,3.429742,3.811231,3.999477,0.226824,1.923647,1.188082,0.294526
min,25.060000,-0.020000,1.000000,-5.380000,-1.000000,-0.050000,-1.000000,-0.050000,-0.060000,-0.100000
25%,53.500000,0.280000,13.340000,0.920000,0.510000,1.540000,0.040000,1.280000,2.640000,0.070000
50%,65.650000,0.540000,14.600000,1.900000,1.490000,3.260000,0.080000,2.870000,3.370000,0.130000
75%,71.650000,0.980000,15.850000,3.800000,4.480000,7.240000,0.150000,4.450000,4.050000,0.240000
max,83.960000,18.800000,39.930000,36.000000,38.640000,34.290000,19.000000,16.060000,11.250000,16.000000


In [85]:
# Identify rows with zero or negative values
invalid_rows = X[(X <= 0).any(axis=1)]

if not invalid_rows.empty:
    print("Dropping rows with invalid values:")
    print(invalid_rows)

# Drop rows with zero or negative values
data_cleaned = X[(X > 0).all(axis=1)]

# # Perform CLR transformation on cleaned data
# clr_data = clr(data_cleaned + 1e-6)  # Adding a small constant to avoid log(0)

# print(clr_data)

Dropping rows with invalid values:
         sio2  tio2  al2o3  fe2o3   mgo   cao   mno   k2o  na2o  p2o5
328     76.52  0.05  12.79   1.27  0.03  0.66  0.03  4.88  3.54 -0.01
329     76.88  0.03  12.79   1.28  0.03  0.64  0.03  4.11  4.15 -0.01
330     76.74  0.04  13.02   1.07  0.05  0.67  0.03  4.84  3.60 -0.01
331     77.38  0.05  12.67   0.98  0.04  0.61  0.02  4.54  3.76 -0.01
332     76.74  0.03  12.65   0.90  0.04  0.60  0.02  5.07  3.34 -0.01
...       ...   ...    ...    ...   ...   ...   ...   ...   ...   ...
145486  64.09  0.24  16.11   6.02  0.00  0.09  0.13  5.88  7.48  0.03
145503  69.63  0.19  13.98   4.61  0.03  0.43  0.13  4.64  6.36  0.00
145504  71.85  0.21  12.28   4.65  0.00  0.46  0.06  4.46  6.02  0.00
145509  74.69  0.19  11.44   3.57  0.01  0.22  0.09  4.34  5.45  0.00
145515  67.65  0.29  12.40   5.52  0.00  1.44  0.20  4.72  6.73  0.06

[1562 rows x 10 columns]


,sio2,tio2,al2o3,fe2o3,mgo,cao,mno,k2o,na2o,p2o5
10,52.030,0.97,18.300,4.260,3.800,8.290,0.13,0.930,3.010,0.200
16,63.640,0.38,15.430,2.160,2.100,4.370,0.14,3.870,3.370,0.250
19,62.860,0.43,18.250,4.250,1.080,0.640,0.08,6.350,5.190,0.070
21,52.262,0.71,18.418,9.056,8.568,9.193,0.17,0.557,1.354,0.079
22,55.930,1.07,15.960,1.950,4.730,7.540,0.19,2.010,1.900,0.200
...,...,...,...,...,...,...,...,...,...,...
145535,65.770,0.55,16.050,1.650,0.270,2.950,0.08,5.630,3.300,0.150
145536,62.390,0.78,16.130,2.790,0.460,3.810,0.11,5.120,3.310,0.230
145537,60.710,1.09,13.890,4.290,0.410,4.400,0.20,4.370,2.830,0.300
145538,63.750,0.69,16.620,2.090,0.280,3.510,0.07,5.570,3.240,0.200


In [ ]:
# pipelines = {
#     'Logistic Regression': Pipeline([
#         ('classifier', LogisticRegression(max_iter=1000))
#     ]),
#     'Random Forest': Pipeline([
#         ('classifier', RandomForestClassifier(random_state=13))
#     ]),
#     'Decision Tree': Pipeline([
#         ('classifier', DecisionTreeClassifier(random_state=13))
#     ]),
#     'KNN': Pipeline([
#         ('classifier', KNeighborsClassifier())
#     ]),
#     'SVM': Pipeline([
#         ('classifier', SVC(random_state=13))
#     ]),
#     'Gradient Boosting': Pipeline([
#         ('classifier', GradientBoostingClassifier(random_state=13))
#     ])
# }

# # Train and evaluate each model
# for model_name, pipeline in pipelines.items():
#     print(f"Evaluating {model_name}...")
    
#     # Train the model on the global dataset
#     pipeline.fit(X_train, y_train)
    
#     # Evaluate on the validation set
#     predictions = pipeline.predict(X_test)
#     print(f"Validation Classification Report for {model_name}:")
#     print(classification_report(y_test, predictions))
    
#     # Confusion matrix for validation set
#     print(f"Confusion Matrix for {model_name}:")
#     print(confusion_matrix(y_test, predictions))
    
#     # Hyperparameter tuning 
#     # Note: You can add hyperparameter tuning as needed for each model
    
#     # Final evaluation on the test set with hyperparameter tuning
# #     y_test_pred = pipeline.predict(X_test)
# #     print(f"Test Classification Report for {model_name}:")
# #     print(classification_report(y_test, y_test_pred))
    
# #     # Confusion matrix for test set
# #     print(f"Confusion Matrix for {model_name}:")
# #     print(confusion_matrix(y_test, y_test_pred))
    
#     print("------------------------------------------")

# Performance measures

## Measuring accuracy using cross-validation

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

scaler = StandardScaler()
model = LogisticRegression(max_iter=1000)
pipeline = make_pipeline(scaler, model)
pipeline.fit(X_train, y_train)

In [ ]:
predictions = pipeline.predict(X_test)
print(f"Validation Classification Report for Logistic Regression:")
print(classification_report(y_test, predictions))

In [ ]:
print(f"Confusion Matrix for Logistic Regression:")
print(confusion_matrix(y_test, predictions))

In [ ]:
from sklearn.model_selection import StratifiedKFold
# from sklearn.base import clone

In [ ]:
scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')

In [ ]:
print("Cross-validation scores:", scores)
print("Mean accuracy:", scores.mean())
print("Standard deviation:", scores.std())

### Ensure that your model is not overfitting by reviewing the complexity of the model and the nature of the data.

In [ ]:
param_grid = {'C': [0.01, 0.1, 1, 10, 100]}
grid_search = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

In [ ]:
# # Step 1: Import Necessary Libraries
# from sklearn.feature_selection import SelectKBest, f_classif
# from sklearn.datasets import load_iris  # Example dataset
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split, cross_val_score

# # Step 2: Load and Prepare Your Data
# data = load_iris()
# X = data.data  # Feature matrix
# y = data.target  # Target vector

# # Step 3: Select Top k Features
# k = 2  # Number of top features to select, you can adjust this
# selector = SelectKBest(f_classif, k=k)
# X_new = selector.fit_transform(X, y)

# # Step 4: Split the Data
# X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42)

# # Step 5: Train and Evaluate Your Model
# model = LogisticRegression(max_iter=1000)
# model.fit(X_train, y_train)
# score = model.score(X_test, y_test)
# print(f"Model accuracy with top {k} features: {score:.4f}")

# # Optional: Cross-Validation
# scores = cross_val_score(model, X_new, y, cv=5, scoring='accuracy')
# print(f"Cross-validation scores with top {k} features: {scores}")
# print(f"Mean accuracy: {scores.mean():.4f}")
# print(f"Standard deviation: {scores.std():.4f}")

# # Optional: Analyzing Selected Features
# selected_features = selector.get_support(indices=True)
# print("Selected feature indices:", selected_features)
# print("Selected feature names:", [data.feature_names[i] for i in selected_features])


In [ ]:
def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=None, train_sizes=np.linspace(0.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    plt.xlabel("Training examples")
    plt.ylabel("Score")

    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)

    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")

    plt.legend(loc="best")
    return plt

# Step 5: Train and Plot Learning Curve
# model = LogisticRegression(max_iter=1000)
title = "Learning Curve (Logistic Regression)"
plot_learning_curve(model, title, X, y, cv=5, n_jobs=-1)

plt.show()